In [ ]:
import pandas as pd
import os
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import time

## Read data

In [ ]:
from src.features import *
train = load_raw()
cols = imply_columns(train)

In [ ]:
assert not train['Unnamed: 0'].duplicated().any()

In [ ]:
train = train.set_index('Unnamed: 0')

In [ ]:
train.shape, train.columns

In [ ]:
# pd.isnull(train[cols['features']]).sum(axis=1).sort_values(ascending=True).head(n=20)
train.shape[0], len(np.where(pd.isnull(train[cols['features']]).sum(axis=1) < 5)[0])
# train.iloc[0,'1972 [YR1972]']
# pd.isnull(train.loc[0,'1972 [YR1972]'])
# train.head()
# train.loc[131876]

## read submission file

In [ ]:
df_submit = pd.read_csv('data/raw/submission.csv').set_index('Unnamed: 0')
df_submit.head()

In [ ]:
len(set(train.loc[df_submit.index]['Country Name']))

In [ ]:
set(train.loc[df_submit.index]['Series Code'])

## preprocess data to backfill nan

In [ ]:
my_rename = {x: x[6:-1] for x in cols['features']}
train.rename(columns=my_rename, inplace=True)

In [ ]:
cols['features'] = list(my_rename.values())
cols['features'].sort()
cols['features'][:5]

In [ ]:
# wide to long format so that years are an index along the rows
train = pd.wide_to_long(train.reset_index(), ["YR"], i="Unnamed: 0", j="year").reset_index(level='year')

In [ ]:
# rename YR to "value"
train.rename(columns={'YR': 'value'}, inplace=True)

In [ ]:
train['year'] = train['year'].apply(lambda x: int(x))
train['year'].head().values

In [ ]:
# plot again just to confirm that wide-to-long worked
country = 'Kenya' # 'Poland'
series = 'Net taxes on products (current LCU)'
subtrain = train[(train['Country Name']==country) & (train['Series Name']==series)]
subtrain = subtrain.set_index('year')
subtrain.head()

In [ ]:
subtrain['value'].head()

In [ ]:
sns.tsplot(subtrain['value'])
plt.title("%s: %s"%(country, series))
plt.show()

## group and backfill

In [ ]:
train.shape

In [ ]:
# test
df = pd.DataFrame([[np.nan, 2, np.nan, 0, 2, 2],
                   [3, 4, np.nan, 1, 1, 2],
                   [np.nan, np.nan, np.nan, 5, 1, 3],
                   [np.nan, 3, np.nan, 4, 1, 4],
                  ],
                  columns=list('ABCDEF'))
df['B'] = df.sort_values('F', ascending=True).groupby(['E'])['B'].fillna(method='ffill')
df

In [ ]:
train.shape

In [ ]:
n_nan_before = pd.isnull(train['value']).sum()
print('before: number of nan: %s'%n_nan_before)

In [ ]:
# around 15 seconds
print(time.ctime(), 'start group')
train2 = train.copy()
train2 = train2.sort_values(['Country Name', 'Series Code', 'year'], ascending=True)
t_group = train2.groupby(['Country Name', 'Series Code'])
print(time.ctime(), 'end group')

In [ ]:
# around 1 minute
print(time.ctime(), 'start fill')
train2['value'] = t_group['value'].fillna(method='ffill')
print(time.ctime(), 'end fill')

In [ ]:
n_nan_after = pd.isnull(train2['value']).sum()
print('after: number of nan: %s'%n_nan_after)

In [ ]:
train2.head()

## check some of the remaining nans

In [ ]:
train2[pd.isnull(train2['value'])].head()

In [ ]:
subtrain = train[(train['Country Name']=='Afghanistan') & (train['Series Code']=='2.1')]
subtrain

In [ ]:
subtrain = train2[(train2['Country Name']=='Afghanistan') & (train2['Series Code']=='2.1')]
subtrain

In [ ]:
plt.plot(train.loc[559].reset_index().set_index('year')['value'], 'b.')
plt.plot(train.loc[618].reset_index().set_index('year')['value'], 'r.')
plt.show()

In [ ]:
for sc in ['2.1', '7.8']:
    # subtrain = train[train['Series Code']=='7.8'].set_index('year')
    subtrain = train2[train2['Series Code']==sc].set_index('year')
    plt.plot(subtrain[subtrain['Country Name']=='Afghanistan']['value'], label='Afghanistan')
    plt.plot(subtrain[subtrain['Country Name']=='Albania']['value'], label='Albania')
    plt.plot(subtrain[subtrain['Country Name']=='Algeria']['value'], label='Algeria')
    plt.title(sc)
    plt.legend()
    plt.show()

## drop the remaining nan

In [ ]:
print(train2.shape)
train2 = train2[~pd.isnull(train2['value'])]
print(train2.shape)

In [ ]:
len(set(train['Series Code'])), len(set(train['Country Name'])), len(set(train.index))

In [ ]:
len(set(train2['Series Code'])), len(set(train2['Country Name'])), len(set(train2.index))

## fit LS for a single country/code pair

In [ ]:
train.loc[df_submit.index[0], ['Series Code', 'Country Name']].head()

In [ ]:
# multi-indexing in pandas
# https://pandas.pydata.org/pandas-docs/stable/advanced.html#
# Note that first usage of train3 will be slow
train3 = train2.reset_index().set_index(['Series Code', 'Country Name'])

In [ ]:
# least squares
# https://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.stats.linregress.html

x = train3['year']['7.8']['Afghanistan'].values
y = train3['value']['7.8']['Afghanistan'].fillna(value=0).values

x, y

In [ ]:
from scipy import stats
import numpy as np
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

slope, intercept, r_value, p_value, std_err

In [ ]:
[slope*year + intercept for year in [2008, 2012]]

In [ ]:
set(train3['Unnamed: 0']['7.8']['Afghanistan'])

In [ ]:
df_submit.loc[559, '2008 [YR2008]']

## Fit for all country/code pairs

In [ ]:
submit_inds[:5]

In [ ]:
train3 = train3.reset_index().set_index("Unnamed: 0")
train3.head()

In [ ]:
df_submit['slope'] = np.nan
df_submit['intercept'] = np.nan
df_submit['r_value'] = np.nan
df_submit['p_value'] = np.nan
df_submit['std_err'] = np.nan

In [ ]:
# least squares
# https://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.stats.linregress.html
n = len(submit_inds)
for j, index in enumerate(submit_inds):
    if j%20==0: print("%s / %s .. %s"%(j, n, index))
    n_avail = len(np.where(train3.index == index)[0])
    if n_avail <= 1:
        print("skipping %s since only %s point available"%(index, n_avail))
        continue

    x = train3.loc[index, 'year'].values
    y = train3.loc[index, 'value'].fillna(value=0)
    y = y.values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    df_submit.loc[index, 'slope'] = slope
    df_submit.loc[index, 'intercept'] = intercept
    df_submit.loc[index, 'r_value'] = r_value
    df_submit.loc[index, 'p_value'] = p_value
    df_submit.loc[index, 'std_err'] = std_err
    df_submit.loc[index, '2008 [YR2008]'] = slope*2008 + intercept
    df_submit.loc[index, '2012 [YR2012]'] = slope*2012 + intercept

## prepare submission file

In [ ]:
df_submit2 = df_submit.copy()
df_submit2 = df_submit2.fillna(value=0)
df_submit2 = df_submit2.reset_index()[['Unnamed: 0', '2008 [YR2008]', '2012 [YR2012]']]
df_submit2.rename(columns={'Unnamed: 0': ''}, inplace=True)
fn1, fn2 = make_submission(df_submit2)
fn1, fn2